In [ ]:
!pip install numpy hist matplotlib mplhep pandas plotly scipy tqdm

In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import rawEventBuilder
import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//"+p, data_list))[:1] #
#file_path = ['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240818_1925.raw']


In [8]:

def plot_alignment(mets, time, order, max_process_event_chunk, file_name):
    fig, ax = plt.subplots(figsize=(10, 8))

    for idx, item in enumerate(order):
        met = mets[idx]
        i, j = item
        binsx = [x for x in range(len(met))]
        ax.plot(time, met, label=f'TDC{i} and TDC{j}, offset 0')
    ax.set_xlim(0, time[-1])
    ax.set_ylim(-1, 40)
    ax.legend()
    ax.set_title(f'Alignment graph {file_name}')
    ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
    ax.set_xlabel('Time (s)')
    plt.savefig(f"output//transition//alignment_{file_name}.png")

def plot_bvg(tdc_mets, time, max_process_event_chunk, file_name):
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    fig, ax = plt.subplots(figsize=(10, 8))
    for tdc in range(5):
        met = tdc_mets[tdc]
        binsx = [x*26.5 for x in range(len(met))]
        ax.plot(time[::25], met, label = f'tdc {tdc}', color = colors[tdc])

    ax.set_xlim(0,time[-1])
    ax.legend()
    ax.set_title(f'TDC monitoring metric {file_name}')
    ax.set_ylabel('bad time behavior / nominal time behavior')
    ax.set_xlabel('Time (s)')
    plt.savefig(f"output//transition//tdc_monitoring{file_name}.png")
    plt.close()


In [7]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
importlib.reload(rawEventBuilder)
import matplotlib.pyplot as plt

interval = 100 # Set your monitoring chunck size
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
file_path  = ["C:\\Users\\jony\\Programming\\Python\\Anubis\\anubis\\data\\proAnubis_240815_1759.raw"]

for file in file_path:
    max_process_event_chunk = 20_000 # End the loop early
    fReader = rawFileReader.fileReader(file) # load in the classs object
    #fReader.skip_events(2000*100) # Skip the first event
    
    processedEvents = 0 # Initialisation
    noiseStart = 0
    firstNoise = False
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    originTime = max([initial_event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if initial_event_chunk[0].tdcEvents[tdc].time])
    last_reset = 0
    mets = [[] for tdc in range(5)]
    time = []
    mets_off = [[] for tdc in range(5)]
    file_name = file.split("//")[-1][:-4]
    tdc_mets = [[] for tdc in range(5)]
    Tot_TDC_info = [[] for tdc in range(5)]
    chunks = []
    bad_chunks = 0

    event_time = max([initial_event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if initial_event_chunk[0].tdcEvents[tdc].time])
    print(event_time)
    while event_time < datetime.strptime("2024-08-15 17:18:30", '%Y-%m-%d %H:%M:%S'):
        fReader.skip_events(2_000)
        initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if initial_event_chunk:
            event_time = max([initial_event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if initial_event_chunk[0].tdcEvents[tdc].time])

    print(event_time)    

    with tqdm(total=max_process_event_chunk, desc=f"Processing Events {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) # get the aligned events
            except Exception as e:
                print("Exception:", e)
                max_process_event_chunk = processedEvents
                break

            event_time = max([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
            
            if last_reset:
                if (event_time - last_reset).total_seconds() > 60:
                    last_reset = event_time
                    fReader.evtBuilder = rawEventBuilder.eventBuilder()
            else:
                last_reset = event_time
                originTime = event_time
                
            [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
            [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
            time.append((event_time-originTime).total_seconds())
            for idx, (i, j) in enumerate(order):
                x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
                alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
                #alignMet_off = ATools.calcAvgAlign(event_chunk, 15, x, y, l, m, i, j, processedEvents) # determine the alignment metric
                
                #mets_off[idx].append(alignMet_off) # write to memory
                mets[idx].append(alignMet) # write to memory
                if alignMet > 40 and not firstNoise:
                    for tdc in range(5):
                        if event_chunk[0].tdcEvents[tdc].time != 0:
                            noiseTime = event_chunk[0].tdcEvents[tdc].time
                            break
                    if noiseTime == 0:
                        print("No hits in the event???")
                    startTime =  noiseTime - originTime
                    print(startTime)
                    noiseStart = processedEvents*interval
                    print(noiseStart)
                    firstNoise = True

            pbar.update(1)
    
    #plot_alignment(mets_off, order, max_process_event_chunk, file_name+"_off")
    plot_alignment(mets, time, order, max_process_event_chunk, file_name+"new")
    #plot_bvg(tdc_mets,max_process_event_chunk, file_name+"fuck")
    plot_bvg(tdc_mets, time, max_process_event_chunk, file_name+"new")
    
    if noiseStart:
        TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel_perEv_{file_name}.pdf')
    else:
        TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel_perEv_{file_name}.pdf')
    

2024-08-15 16:59:49.312501
2024-08-15 17:18:30.477887


Processing Events C:\Users\jony\Programming\Python\Anubis\anubis\data\proAnubis_240815_1759: 100%|██████████| 20000/20000 [05:20<00:00, 62.41Events/s]


NameError: name 'plot_alignment' is not defined

In [11]:
plot_alignment(mets, time, order, max_process_event_chunk, "hellonew")
    #plot_bvg(tdc_mets,max_process_event_chunk, file_name+"fuck")
#plot_bvg(tdc_mets, time[:-1], max_process_event_chunk, "hello"+"new")

#plot_bvg(tdc_mets,max_process_event_chunk, file_name+"fuck")
#plot_bvg(tdc_mets, time, max_process_event_chunk, file_name+"new")
    

ValueError: x and y must have same first dimension, but have shapes (800,) and (801, 1)

In [10]:
plot_alignment(mets[:100_000], order, 100_000, file_name+"HUGEshort")
    #plot_bvg(tdc_mets, max_process_event_chunk, file_name+"fuck")
#plot_bvg(tdc_mets, max_process_event_chunk, file_name+"HUGE")
    
if noiseStart:
    TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel_perEv_{file_name}.pdf')
else:
    TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel_perEv_{file_name}.pdf')
    

KeyboardInterrupt: 

In [13]:
import csv 
with open('alg_huge.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(mets)

In [36]:
TTools.plot_tdc_error_times_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], output_pdf=f'output//transition//Time_Hit_{file_name}.pdf')

In [40]:
#print([mets[i].index(max(mets[i])) for i in range(4)])
import Visual_tools as VTools
importlib.reload(VTools)


#for chunk in chunks:
#    print(VTools.metric_possible(chunk, 2,3))
x = [i for i in range(max_process_event_chunk)]
tot_good1 = [0 for chunk in range(max_process_event_chunk)]
tot_good2 = [0 for chunk in range(max_process_event_chunk)]
tot_poss = [0 for chunk in range(max_process_event_chunk)]
print(len(x))
for i, chunk in enumerate(chunks):
    tot_poss[i], tot_good1[i], tot_good2[i] = VTools.metric_possible(chunk, 2, 3)

plt.plot(x, tot_good1, color='blue')
plt.plot(x, tot_good2, color='red')
plt.show()

plt.plot(x, tot_poss)
plt.show()
#VTools.all_hits(chunks[31], 2, 3, 100)

500


In [33]:
importlib.reload(TTools)
odlozim = Tot_TDC_info.copy()
for tdc in range(5):
    print(len(Tot_TDC_info[tdc]))
    Tot_TDC_info[tdc] = Tot_TDC_info[tdc][:noiseStart]
TTools.plot_tdc_error_channels(Tot_TDC_info, tdcs_to_plot=[2,3])

Tot_TDC_info = odlozim.copy()
print(len(odlozim))
for tdc in range(5):
    print(len(Tot_TDC_info[tdc]))
    Tot_TDC_info[tdc] = Tot_TDC_info[tdc][noiseStart:]
TTools.plot_tdc_error_channels(Tot_TDC_info, tdcs_to_plot=[2,3])

46958
50442
40026
44355
30623
5
46958
50442
40026
44355
30623


In [37]:
importlib.reload(VTools)
counts = [[] for tdc in range(5)]
for chunk in chunks:
    count = VTools.abs_count(chunk)
    for tdc in range(5):
        counts[tdc].append(count[tdc])

#plot it
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    ax.plot(x, counts[tdc], label = f'TDC {tdc}')
ax.legend()
ax.set_title(f'Counts per TDC')
ax.set_ylabel('Counts')
ax.set_xlabel('Processed Event chunks')
plt.savefig(f"output//transition//TDC_counts.png")
plt.show()

In [12]:

fig, ax = plt.subplots(figsize=(10, 8))

for idx, item in enumerate(order):
    met = mets[idx]
    i, j = item
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')
ax.set_xlim(0, max_process_event_chunk)
ax.set_ylim(-1, 40)
ax.legend()

ax.set_title(f'Alignment graph {file_name}')
ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
ax.set_xlabel('Processed Event chunks')
plt.savefig(f"output//transition//alignment_graph{file_name}.png")

colors = ['blue', 'green', 'red', 'purple', 'orange']
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_mets[tdc]
    binsx = [x*26.5 for x in range(len(met))]
    ax.plot(binsx,met, label = f'tdc {tdc}', color = colors[tdc])

ax.set_xlim(0,max_process_event_chunk)
ax.legend()
ax.set_title(f'TDC monitoring metric {file_name}')
ax.set_ylabel('bad time behavior / nominal time behavior')
ax.set_xlabel('processed event chunks')
plt.savefig(f"output//transition//tdc_monitoring{file_name}.png")
plt.close()

#TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, noiseStart), (noiseStart, max_process_event_chunk*interval)], tdcs_to_plot=None, output_pdf=f'output//transition//TDC_channel{file_name}.pdf')
